# ShearLineCNN
> Shear Line Classification using CNN.

## Revision History

| #   | Date       | Action                                           | Modified by        |
|-----|------------|--------------------------------------------------|--------------------|
|     |            |                                                  |                    |
| 015 | 2025-05-22 | Decrease initial LR                              | rmaniego           |
| 014 | 2025-05-22 | Change to Tversky loss                           | rmaniego           |
| 013 | 2025-05-21 | Add data augmentation                            | rmaniego           |
| 012 | 2025-05-21 | Add data preprocessing                           | rmaniego           |
| 011 | 2025-05-20 | Migrate to U-Net architecture                    | rmaniego           |
| 010 | 2025-05-16 | Optimize architecture                            | rmaniego           |
| 009 | 2025-05-16 | Improve architecture                             | rmaniego           |
| 008 | 2025-05-15 | Fix model metrics                                | rmaniego           |
| 007 | 2025-05-03 | Fix testing evaluation                           | rmaniego           |
| 006 | 2025-05-03 | Fix dataset loader                               | rmaniego           |
| 005 | 2025-05-03 | Fix segmentation dataset                         | rmaniego           |
| 004 | 2025-04-10 | Fix architecture to match dataset                | rmaniego           |
| 003 | 2025-04-10 | Update architecture base codes                   | rmaniego           |
| 002 | 2025-04-09 | Prepare dataset                                  | rmaniego           |
| 001 | 2025-03-29 | Create GitHub repository                         | rmaniego           |

## Step 1. Mount Google Drive

**Notes:**.
 - This requires GDrive permissions.
 - Update changes in local repository.
 - Re-run cell for every commit changes in the repository.
 - Colab is read only, unless set in GitHub FGPATs

```python
pip install jupyterlab
pip install notebook
jupyter notebook
```

**GitHub Personal Access Tokens (PAT)**
1. Go to `https://github.com/settings/tokens`.
2. On the sidebar, select `Fine-grained tokens`.
3. Fill-up appropriate details, limit read/write access.
4. Copy generated `PAT` to local environment variables.
5. Do the same to Google Colab secrets.
6. Once expired, move the old repo in GDrive to trash.

In [ ]:
import os

github_fgpat = None
live_on_colab = False
environment_ready = False

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

try:
    from google.colab import drive, userdata

    drive.mount("/content/drive")

    live_on_colab = True
    github_fgpat = userdata.get("ShearLineCNN")
    print("Running on Google Colaboratory...")
except ImportError:
    print("Running locally...")

## Step 2. Check Colab Compute Engine Backend
**Note:** Execute to verify HW accelerator allocation, use information on manuscript.

HW accelerator availability may vary, so ensure that the session is timed and is connected to expected runtime environment in all iterations. Options include:
1. NVIDIA A100 Tensor Core GPU - high-performance deep learning training (recommended).
2. NVIDIA L4 Tensor Core GPU - optimized for AI inference tasks with high performance and efficiency (preferred during HP fine-tuning).
3. NVIDIA T4 Tensor Core GPU - cost-effective, versatile, and suitable for a variety of tasks.


In [ ]:
if live_on_colab:
    gpu_info = !nvidia-smi
    gpu_info = "\n".join(gpu_info)
    if gpu_info.find("failed") >= 0:
        print("Not connected to a GPU")
    else:
        print(gpu_info)

    from psutil import virtual_memory

    ram_gb = virtual_memory().total / 1e9
    print(f"Your runtime has {ram_gb:.1f} gigabytes of available RAM")

    if ram_gb < 20:
        print("Not using a high-RAM runtime")
    else:
        print("You are using a high-RAM runtime!")

## Step 3. Change Working Directory

**Notes:**  
1. Before continuing, make sure you check your GDrive storage usage; cloning on limited storage may impact contents.  
2. Execute cell to ensure the notebook is running under the latest version of project repository.  

In [ ]:
if live_on_colab:
    NB = "/content/drive/MyDrive/Colab Notebooks"
    os.makedirs(NB, exist_ok=True)
    os.chdir(NB)

    def update_repo():

        REPO = f"{NB}/ShearLineCNN2"
        if not os.path.isdir(REPO):
            !git clone https://{github_fgpat}@github.com/rmaniego/ShearLineCNN.git ShearLineCNN2
            os.chdir(REPO)
            return

        os.chdir(REPO)
        !git reset --hard HEAD
        !git pull origin main

    update_repo()

print(os.getcwd())

## Step 4. Install Dependencies
**Note:** Execute cell everytime the `Google Colab` runtime environment reconnected.

In [ ]:
if live_on_colab:
    %pip install -U jupyterlab
    %pip install -U notebook
    %pip install -U opencv-python
    %pip install -U scikit-learn
    %pip install -U scikit-image
    %pip install -U matplotlib
    %pip install -U seaborn
    %pip install -U tensorflow
    %pip install -U tabulate
print("Environment is ready...")

## Step 5: Import the Packages  

import all third party libraries necessary for the ANN model to execute successfully.

In [ ]:
import glob
import json
import time
import random
import warnings
from datetime import datetime

warnings.filterwarnings("ignore", category=RuntimeWarning, message="os.fork()")
warnings.filterwarnings("ignore", category=UserWarning, message="Your `PyDataset` class should call")
warnings.filterwarnings("ignore", category=UserWarning, message="warn")

import cv2
import numpy as np
import tensorflow as tf
from skimage.morphology import disk
from scipy.ndimage import binary_dilation
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping  #, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Conv2DTranspose, UpSampling2D, Concatenate, Cropping2D
from tensorflow.keras.optimizers import Adadelta, AdamW, Lion, RMSprop, SGD
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras import backend as K


gpus = tf.config.list_physical_devices("GPU")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
            print(e)
    print("GPU detected. Running on GPU.")
else:
    print("No GPU detected. Running on CPU.")

## Step 6: Load Datasets  

Load and prepare the training and testing datasets.

In [ ]:
def load_features(source, target, category):
    basenames, features, labels = [], [], []

    sources = glob.glob(f"{source}/{category}/*.json")
    for i, source_path in enumerate(sources):
        filename = os.path.basename(source_path)

        with open(source_path, "r", encoding="utf-8") as file:
            data1 = np.array(json.load(file))

        target_path = f"{target}/{category}/{filename}"
        with open(target_path, "r", encoding="utf-8") as file:
            data2 = np.array(json.load(file))

        features.append(data1)
        labels.append(data2)
        basenames.append(filename)

    return basenames, np.array(features), np.array(labels)


#################################################
# Label Dilation (Widening)                     #
#-----------------------------------------------#
# Dilate shear line annotation to help model    #
# converge faster. Since 1px is too thin        #
# for the model to classify easiliy.            #
#################################################

def label_line_widening(labels_array, target_width=5):
    """
    Modifies the line width in a batch of binary segmentation masks in-memory
    to a specified target width using morphological dilation.
    """

    if target_width < 1 or target_width % 2 == 0:
        raise ValueError("target_width must be a positive odd integer (e.g., 1, 3, 5).")

    dilation_radius = (target_width - 1) // 2

    if dilation_radius == 0:
        return labels_array.astype(labels_array.dtype)

    struct_elem = disk(dilation_radius)

    modified_labels = np.zeros_like(labels_array, dtype=labels_array.dtype)
    num_samples = labels_array.shape[0]

    for i in range(num_samples):
        label_2d = labels_array[i].squeeze()

        modified_label_2d = binary_dilation(label_2d, structure=struct_elem).astype(labels_array.dtype)

        if len(labels_array.shape) == 4:
            modified_labels[i] = np.expand_dims(modified_label_2d, axis=-1)
        else:
            modified_labels[i] = modified_label_2d

    return modified_labels


basenames_list, features_list, labels_list = [], [], []

categories = ["no-shear", "shear"]
segmentation_source = "data/segmentation/source"
segmentation_target = "data/segmentation/target"
for category in categories:
    basenames, features, labels = load_features(segmentation_source, segmentation_target, category)
    if features.size > 0:
        basenames_list.extend(basenames)
        features_list.append(features)
        labels_list.append(labels)

features = np.vstack(features_list)
labels = np.vstack(labels_list)
basenames = np.array(basenames_list)

target_width = 3
labels = labels.astype(np.uint8)
labels = label_line_widening(labels, target_width=target_width)


#################################################
# Perturbation-based Dataset Augmentation       #
#################################################

test_ratio = 0.1
target_total_size = 1000
train_val_target_size = int(target_total_size * (1 - test_ratio))

binary_class_labels = np.array([1 if labels[i].any() else 0 for i in range(len(labels))])

# 90:10 split for testing (using indices) with stratification
total_indices = np.arange(len(basenames))
train_val_indices, test_indices = train_test_split(
    total_indices,
    test_size=test_ratio,
    shuffle=True,
    stratify=binary_class_labels
)

features_train_val = features[train_val_indices]
labels_train_val = labels[train_val_indices]
basenames_train_val = basenames[train_val_indices]

"""
current_train_val_size = len(features_train_val)
if current_train_val_size < train_val_target_size:
    num_augmentations_needed = train_val_target_size - current_train_val_size

    augmented_features_list = []
    augmented_labels_list = []
    augmented_basenames_list = []
    noise_std_dev = 0.5

    idx_to_augment = list(range(current_train_val_size))
    random.shuffle(idx_to_augment)

    aug_count = 0
    while aug_count < num_augmentations_needed:
        if not idx_to_augment:
            idx_to_augment = list(range(current_train_val_size))
            random.shuffle(idx_to_augment)

        original_idx = idx_to_augment.pop(0)

        original_feature = features_train_val[original_idx]
        original_label = labels_train_val[original_idx]
        original_basename = basenames_train_val[original_idx]

        noise = np.random.normal(loc=0.0, scale=noise_std_dev, size=original_feature.shape)
        aug_feature = original_feature + noise
        aug_label = original_label

        augmented_features_list.append(aug_feature)
        augmented_labels_list.append(aug_label)
        augmented_basenames_list.append(f"{original_basename}_aug{aug_count}_noise{noise_std_dev}")

        aug_count += 1
        if aug_count % 100 == 0:
            print(f"Generated {aug_count} augmented samples...")

    augmented_features = np.array(augmented_features_list)
    augmented_labels = np.array(augmented_labels_list)
    augmented_basenames = np.array(augmented_basenames_list)

    features_train_val = np.concatenate((features_train_val, augmented_features), axis=0)
    labels_train_val = np.concatenate((labels_train_val, augmented_labels), axis=0)
    basenames_train_val = np.concatenate((basenames_train_val, augmented_basenames), axis=0)
"""


#################################################
# Dataset Splitting                             #
#################################################

train_val_labels = np.array([1 if labels_train_val[i].any() else 0 for i in range(len(labels_train_val))])

# 80:20 split for train-validation (on train-val set) with stratification
train_val_size = len(features_train_val)
indices_train, indices_val = train_test_split(
    np.arange(train_val_size),
    test_size=0.2,
    shuffle=True,
    stratify=train_val_labels
)

features_train = features_train_val[indices_train]
labels_train = labels_train_val[indices_train]
basenames_train = basenames_train_val[indices_train]

features_val = features_train_val[indices_val]
labels_val = labels_train_val[indices_val]
basenames_val = basenames_train_val[indices_val]

features_test = features[test_indices]
labels_test = labels[test_indices]
basenames_test = basenames[test_indices]

n_train = len(features_train)
n_val = len(features_val)
n_test = len(features_test)

print(f"Train samples: {n_train}")
print(f"Validation samples: {n_val}")
print(f"Test samples: {n_test}")

n_dataset = n_train + n_val + n_test
print(f"TOTAL: {n_dataset}")

print("Dataset ready...")

## Step 7: Define the Architecture  

Define the structure of the convolutional neural network for shear line classification.

In [ ]:
v

### Step 8: Train the Model  

> Feed the training-val dataset to the compiled CNN model.  

**Note:** Re-compile model again before running this step.

In [ ]:
EPOCHS = 1000
BATCH_SIZE = 32  # Increase to 8/16/32/64 in actual run
ARCHITECTURE = "UNet"

MODELS = "models"
ANALYSIS = "analysis"
DATASET = "data"
TEST = "test"

os.makedirs(MODELS, exist_ok=True)
os.makedirs(ANALYSIS, exist_ok=True)

training_timestamp = int(time.time())

early_stopping = EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=5, min_lr=1e-06)  # min α = 1e-5 - 1e-6
history = model.fit(
    features_train,
    labels_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(features_val, labels_val),
    callbacks=[early_stopping, reduce_lr]
)

training_duration = (int(time.time()) - training_timestamp) / 60
print(f"Training completed in {training_duration:.2f} minutes.")


fullpath = f"{MODELS}/shearline.{ARCHITECTURE}_{training_timestamp}.keras"
model.save(fullpath)

with open(f"{ANALYSIS}/metrics_{training_timestamp}.json", "w") as f:
    json.dump({
        "loss": history.history["loss"],
        "tversky_coefficient": history.history["tversky_coefficient"],
        "val_loss": history.history["val_loss"],
        "val_tversky_coefficient": history.history["val_tversky_coefficient"],
        "learning_rate": history.history["learning_rate"]
    }, f, indent=4)

print(f"Model training complete and saved to '{fullpath}'")
print(f"Training and validation metrics saved to '{ANALYSIS}/metrics_{training_timestamp}.json'")

## Step 9: Generate Training Analysis  

**Metrics Definitions**
* Loss is computed based on how far each prediction is from the ground truth, specifically using Dice Loss.
* Dice Coefficient is a specific type of accuracy, focused on the positive class--best for imbalanced datasets.  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


with open(f"{ANALYSIS}/metrics_{training_timestamp}.json", "r") as f:
    metrics = json.load(f)

epochs = range(1, len(metrics["loss"]) + 1)

plt.figure(figsize=(8, 6))
sns.lineplot(x=epochs, y=metrics["loss"], label="Training Tversky Loss", color="blue")
sns.lineplot(x=epochs, y=metrics["val_loss"], label="Validation Tversky Loss", color="orange")
plt.title("Tversky Loss vs. Epochs")
plt.xlabel("Epochs")
plt.ylabel("Tversky Loss")
plt.legend()
plt.grid(True)
plt.savefig(f"{ANALYSIS}/tversky_loss_plot_{training_timestamp}.png")
plt.show()

plt.figure(figsize=(8, 6))
sns.lineplot(x=epochs, y=metrics["tversky_coefficient"], label="Training Tversky Coefficient", color="green")
sns.lineplot(x=epochs, y=metrics["val_v_coefficient"], label="Validation Tversky Coefficient", color="red")
plt.title("Tversky Coefficient vs. Epochs")
plt.xlabel("Epochs")
plt.ylabel("Tversky Coefficient")
plt.legend()
plt.grid(True)
plt.savefig(f"{ANALYSIS}/tversky_coefficient_plot_{training_timestamp}.png")
plt.show()

plt.figure(figsize=(8, 6))
sns.lineplot(x=epochs, y=metrics["learning_rate"], label="Learning Rate", color="violet")
plt.title("Learning Rate vs. Epochs")
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")
plt.legend()
plt.grid(True)
plt.savefig(f"{ANALYSIS}/learning_rate_{training_timestamp}.png")
plt.show()

print(f"\nPlots saved to {ANALYSIS}")

## Step 10: Test the Model

In [ ]:
start_time = time.time()
results = model.predict(features_test, verbose=1)
prediction_duration = time.time() - start_time
image_prediction_time = prediction_duration / len(features_test)

predictions = (results.squeeze(-1) > 0.5).astype("int32")
predictions_flat = predictions.flatten()
labels_test_flat = labels_test.flatten()

print(f"Total prediction time: {prediction_duration:.4f} seconds")
print(f"Time per spatial map: {image_prediction_time:.4f} seconds")

## Step 11: Display the Results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

def visualize_comparison(filename, true_mask, predicted_mask):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(true_mask, cmap="gray")
    plt.title(f"True Mask: {filename}")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(predicted_mask, cmap="gray")
    plt.title(f"Predicted Mask: {filename}")
    plt.axis("off")

    plt.show()

for filename, true_mask, predicted_mask in zip(basenames_test, labels_test, predictions):
    visualize_comparison(filename, true_mask, predicted_mask)

## Step 12: Pixel-wise Segmentation Evaluation

> Evaluate the spatial map accuracy of predicted shear line binary mask using segmentation metrics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def iou(y_true, y_pred):
    intersection = np.logical_and(y_true.astype(bool), y_pred.astype(bool)).sum()
    union = np.logical_or(y_true.astype(bool), y_pred.astype(bool)).sum()
    return intersection / union if union > 0 else 0.0

def dice(y_true, y_pred):
    intersection = np.logical_and(y_true.astype(bool), y_pred.astype(bool)).sum()
    denominator = y_true.astype(bool).sum() + y_pred.astype(bool).sum()
    return 2 * intersection / denominator if denominator > 0 else 0.0

def tversky(y_true, y_pred, alpha=0.7, beta=0.3):
    y_true_f = y_true.astype(bool)
    y_pred_f = y_pred.astype(bool)
    TP = np.logical_and(y_true_f, y_pred_f).sum()
    FP = np.logical_and(np.logical_not(y_true_f), y_pred_f).sum()
    FN = np.logical_and(y_true_f, np.logical_not(y_pred_f)).sum()
    denominator = TP + alpha * FP + beta * FN
    return TP / denominator if denominator > 0 else 0.0

ious = []
dices = []
tverskys = []
accuracies = []

for t_mask, p_mask in zip(labels_test, predictions):
    t_mask = t_mask.reshape(161, 141)
    p_mask = p_mask.reshape(161, 141)

    ious.append(iou(t_mask, p_mask))
    dices.append(dice(t_mask, p_mask))
    tverskys.append(tversky(t_mask, p_mask))
    accuracies.append(np.mean(t_mask.flatten() == p_mask.flatten()))

iou_score = np.mean(ious)
dice_score = np.mean(dices)
tversky_score = np.mean(tverskys)

true_all = np.concatenate([t.flatten() for t in labels_test])
pred_all = np.concatenate([p.flatten() for p in predictions])

cm = confusion_matrix(true_all, pred_all, labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Shear Line", "Shear Line"])

fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(cmap="magma", ax=ax)
plt.title(f"IoU={iou_score:.4f}, Dice={dice_score:.4f}, Tversky={tversky_score:.4f}")
plt.savefig(f"{ANALYSIS}/confusion_matrix_{training_timestamp}.png")
plt.show()

interpretations = {
    (0.90, 1.00): "Excellent",
    (0.75, 0.90): "Good",
    (0.50, 0.75): "Moderate",
    (0.20, 0.50): "Poor",
    (0.00, 0.20): "Very Poor"
}

for score_range, interpretation in interpretations.items():
    if score_range[0] <= iou_score <= score_range[1]:
        print(f"IoU Interpretation: {interpretation}")
        break

for score_range, interpretation in interpretations.items():
    if score_range[0] <= dice_score <= score_range[1]:
        print(f"Dice Interpretation: {interpretation}")
        break

for score_range, interpretation in interpretations.items():
    if score_range[0] <= tversky_score <= score_range[1]:
        print(f"Tversky Interpretation: {interpretation}")
        break

# Step 13: Duplicate Notebook  

**Note:** Manually save first before duplicating the notebook.

In [ ]:
VALIDATIONS = "validations"
os.makedirs(VALIDATIONS, exist_ok=True)

filename = "ShearLineCNN.ipynb"
with open(filename, "r", encoding="utf-8") as src:
    contents = src.read()
    checkpoint = f"{VALIDATIONS}/{filename}".replace(".ipynb", f"_{training_timestamp}.ipynb")
    with open(checkpoint, "w", encoding="utf-8") as dest:
        dest.write(contents)
        print(f"Checkpoint was created at '{checkpoint}'.")

> End of code.